This notebook helps converting Plugin's code into a documentation page. Namely, it coverts the code responsible for config options and dtypes into html tables

In [188]:
import helix as hx

## Options table

Copy code containing config options from the plugin source code into the cell below

In [242]:
config = \
'''
    record_length = strax.Config(
        default=hx.DEFAULT_RECORD_LENGTH, track=False, type=int,
        help='Number of samples per raw_record'
    )
    sampling_dt = strax.Config(
        default=hx.DEFAULT_SAMPLING_DT, track=False, type=int,
        help='Sampling length in ns'
    )
    daq_data_directory = strax.Config(
        default='toy_data', track=False, type=str,
        help='Path to the directory containing run directories with the raw data'
    )
    channel_map = strax.Config(
        default=hx.DEFAULT_CHANNEL_MAP, track=False, type=dict,
        help='Dictionary mapping channel types (SUBMERGED, VACUUM) to the corresponding range of channel numbers'
    )
    run_metadata_directory = strax.Config(
        default='', track=False, type=str,
        help='Path to the directory containing run json metadata'
    )
'''

In [243]:
def get_row(option):
    if not('default' in option and 'type' in option and 'help' in option):
        return ''
    name = option.split('=')[0].strip()
    t = option.split('type=')[1].split(',')[0]
    default = option.split('default=')[1].split(',')[0]
    if 'hx.' in default:
        default = str(eval(default))
    description = ''.join([str(o)+' ' for o in option.split("help='")[1].split("'") if not o.isspace()])
    return \
f"""
    <tr>
        <td><tt>{name}</tt></td><td>{t}</td><td>{default}</td><td>{description}</td>
    </tr>"""

In [244]:
table = \
'''
<table>
    <thead align=left>
        <tr><th>option</th><th>type</th><th>default</th><th>description</th></tr>
    </thead>'''

for option in config.split(')\n'):
    if not option.isspace():
        table += get_row(option)
        
table += \
'''
</table>'''

Copy the output below into the documentation md page

In [245]:
print(table)


<table>
    <thead align=left>
        <tr><th>option</th><th>type</th><th>default</th><th>description</th></tr>
    </thead>
    <tr>
        <td><tt>record_length</tt></td><td>int</td><td>1250000</td><td>Number of samples per raw_record </td>
    </tr>
    <tr>
        <td><tt>sampling_dt</tt></td><td>int</td><td>4000</td><td>Sampling length in ns </td>
    </tr>
    <tr>
        <td><tt>daq_data_directory</tt></td><td>str</td><td>'toy_data'</td><td>Path to the directory containing run directories with the raw data </td>
    </tr>
    <tr>
        <td><tt>channel_map</tt></td><td>dict</td><td>{<ChannelType.SUBMERGED: 0>: (1, 35), <ChannelType.VACUUM: 1>: (36, 50)}</td><td>Dictionary mapping channel types (SUBMERGED, VACUUM) to the corresponding range of channel numbers </td>
    </tr>
    <tr>
        <td><tt>run_metadata_directory</tt></td><td>str</td><td>''</td><td>Path to the directory containing run json metadata </td>
    </tr>
</table>


## dtype table

Look for infer_dtype method or dtype property in the plugin's source code to find its dtype. Copy the dtype code into the cell below. Run the following cells

In [246]:
dtype = \
'''
time_dt_fields + [
        (('Channel number', 'channel'), np.int16),
        (('Channel type', 'channel_type'), np.int16),
        (('Id of the block of records in the run', 'block_id'), np.int32),
        (('Record data in raw ADC counts', 'data'), np.int16, record_length),
    ]'''

In [247]:
def process_time_fields(time_fields):
    if 'time_dt_fields' in time_fields:
        return """
        (('Start time since unix epoch [ns]', 'time'), np.int64),
        (('Length of the interval in samples', 'length'), np.int64),
        (('Width of one sample [ns]', 'dt'), np.int16),""".split('((')
    else:
        return """
        (('Start time since unix epoch [ns]', 'time'), np.int64),
        (('Exclusive end time since unix epoch [ns]', 'endtime'), np.int64),""".split('((')        
    

def get_dtype_row(field):
    description = field.split(',')[0].strip("'")
    name = field.split(',')[1].split("')")[0].strip().strip("'")
    t = field.split("), np.")[1]
    if '\n' in t:
        t = t.split('\n')[0]
    if len(t.strip().strip('),')) > 10:
        shape = ','.join(t.split(',')[1:]).strip(', ')
        if shape[-2:]=='))':
            shape = shape[:-1]
        if shape[0]!='(':
            shape = '(' + shape
        t = t.split(',')[0]
    else:
        shape = '-'
        t = t.strip().strip('),')
    return \
f"""
    <tr>
        <td><tt>{name}</tt></td><td>{t}</td><td>{shape}</td><td>{description}</td>
    </tr>"""

In [248]:
time_fields = dtype.split('+')[0].strip()
other_fields = dtype.split('+')[1].strip(' [').split('((')

fields = process_time_fields(time_fields) + other_fields

table = \
'''
<table>
    <thead align=left>
        <tr><th>field</th><th>type</th><th>shape</th><th>description</th></tr>
    </thead>'''

for field in fields:
    if not field.isspace():
        table += get_dtype_row(field)
        
table += \
'''
</table>'''

Copy the output below into the documentation md page

In [249]:
print(table)


<table>
    <thead align=left>
        <tr><th>field</th><th>type</th><th>shape</th><th>description</th></tr>
    </thead>
    <tr>
        <td><tt>time</tt></td><td>int64</td><td>-</td><td>Start time since unix epoch [ns]</td>
    </tr>
    <tr>
        <td><tt>length</tt></td><td>int64</td><td>-</td><td>Length of the interval in samples</td>
    </tr>
    <tr>
        <td><tt>dt</tt></td><td>int16</td><td>-</td><td>Width of one sample [ns]</td>
    </tr>
    <tr>
        <td><tt>channel</tt></td><td>int16</td><td>-</td><td>Channel number</td>
    </tr>
    <tr>
        <td><tt>channel_type</tt></td><td>int16</td><td>-</td><td>Channel type</td>
    </tr>
    <tr>
        <td><tt>block_id</tt></td><td>int32</td><td>-</td><td>Id of the block of records in the run</td>
    </tr>
    <tr>
        <td><tt>data</tt></td><td>int16</td><td>(record_length)</td><td>Record data in raw ADC counts</td>
    </tr>
</table>
